In [2]:
import spotipy
import requests as req
from datetime import datetime
from pytz import timezone
import json
import os
from time import sleep

In [3]:
CLIENT_ID="111c2079a7a449269f04022acd654cd6"
CLIENT_SECRET="55f544db1a25401db43bb05dfff92635"
Scope="user-read-currently-playing"
redirect_uri='http://localhost:3000'

In [4]:
def create_spotify(CLIENT_ID,CLIENT_SECRET,Scope,redirect_uri):
    auth_manager = spotipy.SpotifyOAuth(
        scope=Scope,
        # username=USERNAME,
        redirect_uri=redirect_uri,
        client_id=CLIENT_ID,
        client_secret=CLIENT_SECRET,open_browser=True)
    spotify = spotipy.Spotify(auth_manager=auth_manager)
    
    return auth_manager, spotify

In [5]:
auth_manager,sp=create_spotify(CLIENT_ID,CLIENT_SECRET,Scope,redirect_uri)

In [6]:
token_info =auth_manager.cache_handler.get_cached_token()
token_info

{'access_token': 'BQAMtBEDAf37VTH1pdO8Avr14O3b45Pvh9CTQ7mCeQtvUDbvTC98sjKxk1Xj8Au6R60q15d3CLRYJ6SntzIymn6vL4ctXdaT720IPKb4vGNqahVOjBLDwzRhtI8K5Zie8duaNbzNCY_OfVgTEwUx0s3kE7yM6Zvi95_V5-1ZGtOMh3yFA58tb5r0ncmNy27p5SUoxapqQDA',
 'token_type': 'Bearer',
 'expires_in': 3600,
 'scope': 'user-read-currently-playing',
 'expires_at': 1718530056,
 'refresh_token': 'AQCS4Ef0sNzb2PKa2sDh9_ZxA7yHkkZ3WBuemnFhFSQ0oyJW1WDPbTcnbNtD-GLi-bvY-_TMGZCmpflBNNM25z8b0STHrwcsd8l51eXBvMrb-Phkr1lvQPwsQCZOPH1P2Tw'}

In [7]:
request=req.get("https://api.spotify.com/v1/me/player/currently-playing",
    headers={"Authorization":f"Bearer {token_info["access_token"]}"
})
print(request)

<Response [200]>


In [8]:
res=sp.currently_playing()
print(res)

{'timestamp': 1718529187567, 'context': None, 'progress_ms': 299035, 'item': {'album': {'album_type': 'single', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/0LyfQWJT6nXafLPZqxe9Of'}, 'href': 'https://api.spotify.com/v1/artists/0LyfQWJT6nXafLPZqxe9Of', 'id': '0LyfQWJT6nXafLPZqxe9Of', 'name': 'Various Artists', 'type': 'artist', 'uri': 'spotify:artist:0LyfQWJT6nXafLPZqxe9Of'}], 'available_markets': ['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DK', 'DO', 'DE', 'EC', 'EE', 'SV', 'FI', 'FR', 'GR', 'GT', 'HN', 'HK', 'HU', 'IS', 'IE', 'IT', 'LV', 'LT', 'LU', 'MY', 'MT', 'MX', 'NL', 'NZ', 'NI', 'NO', 'PA', 'PY', 'PE', 'PH', 'PL', 'PT', 'SG', 'SK', 'ES', 'SE', 'CH', 'TW', 'TR', 'UY', 'US', 'GB', 'AD', 'LI', 'MC', 'ID', 'JP', 'TH', 'VN', 'RO', 'IL', 'ZA', 'SA', 'AE', 'BH', 'QA', 'OM', 'KW', 'EG', 'MA', 'DZ', 'TN', 'LB', 'JO', 'PS', 'IN', 'BY', 'KZ', 'MD', 'UA', 'AL', 'BA', 'HR', 'ME', 'MK', 'RS', 'SI', 'KR', 'BD', 'PK', 'LK', 'GH',

In [9]:
def IST_Time():
    ind_timezone= datetime.now(timezone("Asia/Kolkata"))
    ind_time=ind_timezone.strftime('%H_%M_%S')
    ind_date=ind_timezone.strftime('%Y-%m-%d')
    return ind_date,ind_time


def save_file(res):
    response_object=json.dumps(res)
    ind_date,ind_time=IST_Time()  #('2024-06-16', '13_33_56')
    file_dir=f"./currently_playing/ref=_{ind_date}/"
    if not os.path.exists(file_dir):
        # print("yes")
        os.makedirs(file_dir)
    with open(f"{file_dir}{ind_time}.json",'w') as outfile:
        outfile.write(response_object)

In [10]:
ind_date,ind_time=IST_Time()


In [11]:
latest_track_id=None
while True:
    ind_date,ind_time=IST_Time()
    try:
        print("server Running!!",end="\t")
        res=sp.currently_playing()
        if not res: #
            print(f"User Offline!! {ind_date} {ind_time.replace("_",":")}")
        elif res.get("is_playing")==False:
            print(f"User Paused!! {ind_date} {ind_time.replace("_",":")}")
        else:
            print(f"User Online!! {ind_date} {ind_time.replace("_",":")}")
            current_track_id=res.get("item").get("id")
            if latest_track_id!=current_track_id:
                latest_track_id=current_track_id #update the latest_track_id
                save_file(res)
        sleep(10)
    except:
        print(f"Error occured!!!  {ind_date} {ind_time.replace("_",":")}")
        






    

server Running!!	User Paused!! 2024-06-16 14:45:52
server Running!!	User Paused!! 2024-06-16 14:46:02
server Running!!	User Online!! 2024-06-16 14:46:12
server Running!!	User Online!! 2024-06-16 14:46:22
server Running!!	User Online!! 2024-06-16 14:46:32
server Running!!	User Online!! 2024-06-16 14:46:43
server Running!!	User Online!! 2024-06-16 14:46:53
server Running!!	User Online!! 2024-06-16 14:47:03
server Running!!	User Paused!! 2024-06-16 14:47:13
server Running!!	User Paused!! 2024-06-16 14:47:23
server Running!!	User Offline!! 2024-06-16 14:47:34
server Running!!	User Offline!! 2024-06-16 14:47:44
server Running!!	User Offline!! 2024-06-16 14:47:54
server Running!!	User Offline!! 2024-06-16 14:48:04
server Running!!	User Offline!! 2024-06-16 14:48:14
server Running!!	User Offline!! 2024-06-16 14:48:24
server Running!!	User Offline!! 2024-06-16 14:48:34
server Running!!	User Offline!! 2024-06-16 14:48:44
server Running!!	User Offline!! 2024-06-16 14:48:55
server Running!!	User 